## Movie Recommendation System
* Author: Bhavik Jikadara


##### Description: 
* Recommendation systems are among the most popular applications of data science. They are used to predict the Rating or Preference that a user would give to an item.

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

### Data Collection and Pre-processing

In [2]:
# loading the data from the csv file to a pandas DataFrame
movies_data = pd.read_csv('DataSet/movies.csv')

In [3]:
# printing the first 5 row of the Dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
# numbers of raws and columns in the dataframe
movies_data.shape

(4803, 24)

#### Features Selecting

In [5]:
# selecting the relevant features for recommendation
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [6]:
# replacing the null values with null string
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

In [7]:
# combining all the five selected features
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' '+ movies_data['cast'] + ' ' + movies_data['director']
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [8]:
#  converting the text data to feature vector

vectorizer = TfidfVectorizer(min_df=3,  max_features=None,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

In [9]:
feature_vectorizer = vectorizer.fit_transform(combined_features)

In [10]:
print(feature_vectorizer)

  (0, 3886)	0.1536976627444109
  (0, 225)	0.18566681007004
  (0, 63)	0.16513626207193818
  (0, 5475)	0.19643314443402027
  (0, 7545)	0.1879772273153404
  (0, 10395)	0.199978809874362
  (0, 10073)	0.1683917273809145
  (0, 12028)	0.1780279742723308
  (0, 9628)	0.1933056458677492
  (0, 2496)	0.18566681007004
  (0, 9742)	0.08493477593853131
  (0, 3885)	0.1536976627444109
  (0, 219)	0.13216499401645038
  (0, 57)	0.10202781769968425
  (0, 1544)	0.14121763394895187
  (0, 5467)	0.09222715512680034
  (0, 9276)	0.15880572278967064
  (0, 7540)	0.13129432428633436
  (0, 6462)	0.18566681007004
  (0, 10386)	0.1238704940983226
  (0, 11692)	0.16223668806151217
  (0, 10072)	0.1683917273809145
  (0, 9603)	0.1780279742723308
  (0, 12026)	0.16513626207193818
  (0, 11912)	0.1933056458677492
  :	:
  (4801, 9960)	0.4733738406324099
  (4801, 3492)	0.4197645343341914
  (4801, 298)	0.30110392059547253
  (4801, 8440)	0.3661552280359729
  (4801, 7998)	0.22936895135003765
  (4801, 2590)	0.5677777777094356
  (4802,

### Cosine Similarity

In [11]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectorizer)

In [12]:
print(similarity)

[[1.         0.0377224  0.03626372 ... 0.         0.         0.        ]
 [0.0377224  1.         0.01132382 ... 0.00725447 0.         0.        ]
 [0.03626372 0.01132382 1.         ... 0.         0.07565997 0.        ]
 ...
 [0.         0.00725447 0.         ... 1.         0.         0.02528798]
 [0.         0.         0.07565997 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02528798 0.         1.        ]]


In [13]:
#  getting the movie name from the user
movie_name = input('Enter the favourite movie name: ')

# creating a list with all movie names given in the dataset
list_of_all_title = movies_data['title'].tolist()

# finding the close match for the movie name given by the user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_title)
print(find_close_match)

# close match
close_match = find_close_match[0]
print(close_match)

# finding the index of the movie with title
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

# Getting a list of similar movies
similarity_movies = list(enumerate(similarity[index_of_the_movie]))
len(similarity_movies)

# sorting the movies based on their similarity movies
sorted_similar_movies = sorted(similarity_movies, key = lambda x: x[1], reverse=True)
# print(sorted_similar_movies)

# print the name of similar movies based on the index
print('Movies suggested for you: \n')

i = 1
for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]['title'].values[0]
    if i < 30:
        print(i, '. ', title_from_index)
        i +=1

['Avatar']
Avatar
0
Movies suggested for you: 

1 .  Avatar
2 .  Aliens
3 .  Guardians of the Galaxy
4 .  Beastmaster 2: Through the Portal of Time
5 .  Cargo
6 .  Alien
7 .  X-Men: Days of Future Past
8 .  Star Wars: Clone Wars: Volume 1
9 .  Star Trek Beyond
10 .  Dragonball Evolution
11 .  Superman
12 .  Clash of the Titans
13 .  Galaxy Quest
14 .  Superman III
15 .  Man of Steel
16 .  Superman II
17 .  Jason X
18 .  Space Dogs
19 .  BloodRayne
20 .  Backmask
21 .  Star Trek Into Darkness
22 .  Alien³
23 .  Jumper
24 .  Steamboy
25 .  Moonraker
26 .  Pocahontas
27 .  Hellboy II: The Golden Army
28 .  Gettysburg
29 .  Zathura: A Space Adventure


## Movie Recommendation System

In [14]:
def Movie_Recommendation_System(movie_name):
    # creating a list with all movie names given in the dataset
    list_of_all_title = movies_data['title'].tolist()

    # finding the close match for the movie name given by the user
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_title)

    # close match
    close_match = find_close_match[0]

    # finding the index of the movie with title
    index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

    # Getting a list of similar movies
    similarity_movies = list(enumerate(similarity[index_of_the_movie]))

    # sorting the movies based on their similarity movies
    sorted_similar_movies = sorted(similarity_movies, key = lambda x: x[1], reverse=True)

    # print the name of similar movies based on the index
    print('Movies suggested for you: \n')

    i = 1
    for movie in sorted_similar_movies:
        index = movie[0]
        title_from_index = movies_data[movies_data.index == index]['title'].values[0]
        if i <= 10:
            print(i, '. ', title_from_index)
            i +=1

name = input('Enter the favourite movie name: ')
sys = Movie_Recommendation_System(name)



Movies suggested for you: 

1 .  Avatar
2 .  Aliens
3 .  Guardians of the Galaxy
4 .  Beastmaster 2: Through the Portal of Time
5 .  Cargo
6 .  Alien
7 .  X-Men: Days of Future Past
8 .  Star Wars: Clone Wars: Volume 1
9 .  Star Trek Beyond
10 .  Dragonball Evolution
